# Flight Delay Analysis: Major US Airports 2015
## Introduction
This analysis examines flight delay patterns and airline performance across major US airports (BOS, JFK, SFO, and LAX) using the Department of Transportation's 2015 dataset. We focus on data manipulation using Python's core data science libraries to derive meaningful insights about flight operations and delays.

## Data Preparation
### 1. Initial Setup

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Set file paths
base_path = 'assets/'

# Load datasets
flights_df_raw = pd.read_csv(f'{base_path}flights.csv', low_memory=False)
airports_df = pd.read_csv(f'{base_path}airports.csv')
airlines_df = pd.read_csv(f'{base_path}airlines.csv')

### 2. Data Preprocessing Function
Our core preprocessing function handles several key tasks:

Filtering specific airports (BOS, JFK, SFO, LAX)
Handling missing values
Converting data types
Creating derived features

In [3]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

base_path = 'assets/'

# Load data with correct file paths
flights_df_raw = pd.read_csv(f'{base_path}flights.csv', low_memory=False)
airports_df = pd.read_csv(f'{base_path}airports.csv')
airlines_df = pd.read_csv(f'{base_path}airlines.csv')

def data_preprocess(flights_df):
    """
    Data preprocessing function exactly as shown in the reference
    """
    # Remove rows with missing values
    flights_df = flights_df.dropna()

    # Filter by specific airports
    airports = ['BOS', 'JFK', 'SFO', 'LAX']
    flights_df = flights_df[flights_df['ORIGIN_AIRPORT'].isin(airports)]

    # Filter out flights with more than 1 day delay
    flights_df = flights_df[flights_df['DEPARTURE_DELAY'] <= 1440]

    # Convert FLIGHT_NUMBER to string
    flights_df['FLIGHT_NUMBER'] = flights_df['FLIGHT_NUMBER'].astype(str)

    # Convert SCHEDULED_DEPARTURE to datetime
    flights_df['SCHEDULED_DEPARTURE'] = pd.to_datetime(
        flights_df[['YEAR', 'MONTH', 'DAY']].astype(str).agg('-'.join, axis=1) +
        ' ' +
        (flights_df['SCHEDULED_DEPARTURE'] // 100).astype(int).astype(str).str.zfill(2) +
        ':' +
        (flights_df['SCHEDULED_DEPARTURE'] % 100).astype(int).astype(str).str.zfill(2)
    )

    # Add IS_DELAYED column
    flights_df['IS_DELAYED'] = flights_df['DEPARTURE_DELAY'] >= 15

    # Drop unnecessary columns
    flights_df.drop(['YEAR', 'MONTH', 'DAY'], axis=1, inplace=True)

    return flights_df

# Preprocess the flight data and print results
flights_df = data_preprocess(flights_df_raw.copy())
print("\nQuestion 1 Results:")
print(f"DataFrame Shape after cleaning: {flights_df.shape}")
print("\nFirst few rows of cleaned data:")
print(flights_df.head())


Question 1 Results:
DataFrame Shape after cleaning: (535744, 7)

First few rows of cleaned data:
  ORIGIN_AIRPORT DESTINATION_AIRPORT AIRLINE FLIGHT_NUMBER  \
0            LAX                 PBI      AA          2336   
1            SFO                 CLT      US           840   
2            LAX                 MIA      AA           258   
4            SFO                 MSP      DL           806   
6            LAX                 CLT      US          2013   

  SCHEDULED_DEPARTURE  DEPARTURE_DELAY  IS_DELAYED  
0 2015-01-01 00:10:00             -8.0       False  
1 2015-01-01 00:20:00             -2.0       False  
2 2015-01-01 00:20:00             -5.0       False  
4 2015-01-01 00:25:00             -5.0       False  
6 2015-01-01 00:30:00             14.0       False  


## Analysis Components
### 1. Data Cleaning and Initial Analysis
Objective: Process the raw flight data and establish our baseline dataset.

In [4]:
flights_df = data_preprocess(flights_df_raw.copy())
print(f"DataFrame Shape after cleaning: {flights_df.shape}")
print("\nFirst few rows of cleaned data:")
print(flights_df.head())

DataFrame Shape after cleaning: (535744, 7)

First few rows of cleaned data:
  ORIGIN_AIRPORT DESTINATION_AIRPORT AIRLINE FLIGHT_NUMBER  \
0            LAX                 PBI      AA          2336   
1            SFO                 CLT      US           840   
2            LAX                 MIA      AA           258   
4            SFO                 MSP      DL           806   
6            LAX                 CLT      US          2013   

  SCHEDULED_DEPARTURE  DEPARTURE_DELAY  IS_DELAYED  
0 2015-01-01 00:10:00             -8.0       False  
1 2015-01-01 00:20:00             -2.0       False  
2 2015-01-01 00:20:00             -5.0       False  
4 2015-01-01 00:25:00             -5.0       False  
6 2015-01-01 00:30:00             14.0       False  


### 2. Airport Flight Volume Analysis
Objective: Determine the number of flights operating from each airport.

In [5]:
# Calculate Number of Flights per Airport
def flights_per_airport(flights_df, airports_df):
    flights_df = data_preprocess(flights_df)

    # Merge with airports data
    merged_df = flights_df.merge(airports_df, left_on='ORIGIN_AIRPORT', right_on='IATA_CODE', how='inner')

    # Calculate the number of flights per airport
    num_flights_df = merged_df.groupby('IATA_CODE').size().reset_index(name='NUM_FLIGHTS')

    # Reindex to get counts for specific airports
    num_flights_df.set_index('IATA_CODE', inplace=True)
    num_flights_df = num_flights_df.reindex(['BOS', 'JFK', 'SFO', 'LAX']).reset_index()
    num_flights_df = num_flights_df.set_index(num_flights_df.columns[0])

    return num_flights_df

# Get the number of flights per airport
num_flights_df = flights_per_airport(flights_df_raw.copy(), airports_df.copy())
print("\nQuestion 2 Results:")
print(f"Number of flights per airport:\n{num_flights_df}")


Question 2 Results:
Number of flights per airport:
           NUM_FLIGHTS
IATA_CODE             
BOS             105276
JFK              91933
SFO             145951
LAX             192584


### 3. Airline Performance Analysis
Objective: Identify and analyze the top three airlines based on flight volume and delay metrics.

In [7]:
# Identify Top Three Airlines by Number of Flights and Delay Percentage
def top_three_airlines(flights_df, airlines_df):
    flights_df = data_preprocess(flights_df)

    # Calculate number of flights per airline
    num_flights = flights_df.groupby('AIRLINE')['FLIGHT_NUMBER'].count().reset_index()
    num_flights.columns = ['AIRLINE', 'NUM_FLIGHTS']

    # Calculate number of delayed flights per airline
    num_delayed = flights_df[flights_df['IS_DELAYED']].groupby('AIRLINE')['FLIGHT_NUMBER'].count().reset_index()
    num_delayed.columns = ['AIRLINE', 'NUM_DELAYED']

    # Merge the counts
    merged_df = num_flights.merge(num_delayed, on='AIRLINE', how='left')
    merged_df['NUM_DELAYED'] = merged_df['NUM_DELAYED'].fillna(0)

    # Calculate percentage of delays
    merged_df['PERC_DELAY'] = (merged_df['NUM_DELAYED'] / merged_df['NUM_FLIGHTS']) * 100

    # Sort and get top three airlines
    sorted_df = merged_df.sort_values(['NUM_FLIGHTS', 'PERC_DELAY'], ascending=[False, True])
    top_three_airlines_df = sorted_df.head(3)

    # Merge to get airline names
    top_three_airlines_df = top_three_airlines_df.merge(airlines_df, left_on='AIRLINE', right_on='IATA_CODE', how='left')
    top_three_airlines_df = top_three_airlines_df[['AIRLINE_y', 'NUM_FLIGHTS', 'PERC_DELAY']]
    top_three_airlines_df.columns = ['AIRLINE_NAME', 'NUM_FLIGHTS', 'PERC_DELAY']

    return top_three_airlines_df

# Get top three airlines
top_three_airlines_df = top_three_airlines(flights_df_raw.copy(), airlines_df.copy())
print("\nQuestion 3 Results:")
print("Top three airlines:")
print(top_three_airlines_df)

# Verify the assertions
columns_check = sorted(list(top_three_airlines_df.columns)) == sorted(['NUM_FLIGHTS', 'PERC_DELAY', 'AIRLINE_NAME'])
top_airline_check = top_three_airlines_df.loc[0, 'AIRLINE_NAME'] == 'United Air Lines Inc.'



Question 3 Results:
Top three airlines:
             AIRLINE_NAME  NUM_FLIGHTS  PERC_DELAY
0   United Air Lines Inc.        86562   23.265405
1         JetBlue Airways        85920   20.798417
2  American Airlines Inc.        77024   15.351059


### 4. Monthly Delay Pattern Analysis
Objective: Calculate and visualize monthly delay percentages for each airport.

In [8]:
# Calculate Monthly Percentage of Delays per Airport
def monthly_airport_delays(flights_df):
    flights_df = data_preprocess(flights_df)
    flights_df['SCHEDULED_DEPARTURE'] = pd.to_datetime(flights_df['SCHEDULED_DEPARTURE'])
    flights_df['MONTH'] = flights_df['SCHEDULED_DEPARTURE'].dt.strftime('%B')
    
    # Calculate mean delay for each origin airport and month
    monthly_delays = flights_df.groupby(['ORIGIN_AIRPORT', 'MONTH'])['IS_DELAYED'].mean().round(4).reset_index()
    monthly_delays_pivot = monthly_delays.pivot(index='MONTH', columns='ORIGIN_AIRPORT', values='IS_DELAYED')
    monthly_delays_df = monthly_delays_pivot.reset_index()

    # Sort months in the correct order while keeping as object type
    month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 
                   'October', 'November', 'December']
    
    # Create a temporary categorical for sorting
    monthly_delays_df['MONTH'] = pd.Categorical(monthly_delays_df['MONTH'], categories=month_order, ordered=True)
    monthly_delays_df = monthly_delays_df.sort_values('MONTH')
    
    # Convert back to object type after sorting
    monthly_delays_df['MONTH'] = monthly_delays_df['MONTH'].astype(str)
    
    monthly_delays_df.reset_index(drop=True, inplace=True)

    return monthly_delays_df

# Get monthly percentage of delays per airport
monthly_airport_delays_df = monthly_airport_delays(flights_df_raw.copy())
print("\nQuestion 4 Results:")
print("Monthly percentage of delays per airport:")
print(monthly_airport_delays_df)
print("\nMONTH column dtype:", monthly_airport_delays_df['MONTH'].dtype)


Question 4 Results:
Monthly percentage of delays per airport:
ORIGIN_AIRPORT      MONTH     BOS     JFK     LAX     SFO
0                 January  0.1902  0.2257  0.1738  0.2001
1                February  0.3248  0.3174  0.1978  0.2222
2                   March  0.1984  0.2736  0.2246  0.1770
3                   April  0.1553  0.2020  0.1855  0.1756
4                     May  0.1552  0.1552  0.1990  0.2297
5                    June  0.2032  0.1938  0.2474  0.2374
6                    July  0.1968  0.2019  0.2772  0.2430
7                  August  0.1988  0.1985  0.2289  0.2118
8               September  0.1274  0.1506  0.1486  0.1399
9                November  0.1177  0.1383  0.1741  0.1565
10               December  0.1982  0.2563  0.2627  0.2990

MONTH column dtype: object


## Key Findings
### Flight Volume Distribution:
Analysis of busiest airports
Operational patterns
### Airline Performance Metrics:
Top carriers by volume
Efficiency rankings
Delay statistics
### Temporal Patterns:
Monthly delay variations
Seasonal trends
Airport-specific characteristics
### Conclusions
Summary of major findings
Operational implications
Potential areas for further analysis